In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [3]:
import fitz  # PyMuPDF

def load_pdf(file_path):
    """Load a PDF file and return its content page by page."""
    try:
        # Open the PDF file
        doc = fitz.open(file_path)
        content = []
        
        # Iterate through the pages
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)  # Load each page
            text = page.get_text()  # Extract text from the page
            content.append(text)
        
        doc.close()
        return content
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [4]:
local_path = "act-print-11-BD.pdf"
data = load_pdf(local_path)

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [7]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.create_documents(data)

In [8]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████| 200/200 [07:32<00:00,  2.26s/it]


In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [22]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [23]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [24]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [25]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
chain.invoke(input(""))

ollama pull mistral


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


' The text provided appears to be excerpts from The Penal Code, 1860 of Bangladesh and the Indian Penal Code (IPC). They discuss the definitions and explanations for theft and marriage-related offenses.\n\nIn the context of your question "ollama pull mistral", it seems unclear how this phrase relates to the provided text. If you are asking about the definition of theft or related terms in the text, here\'s a summary:\n\n* Theft is defined as dishonestly moving something capable of being moved with the intention of permanently depriving the owner of it without their consent.\n* Explanation 1 mentions that anything attached to the earth is not subject to theft until it is severed from the earth.\n* Explanation 2 states that if the same act causes both severance and movement, it can be considered theft.\n* Explanation 3 explains that a person can cause an item to move by removing an obstacle or separating it from another thing.\n* Explanation 4 notes that if a person causes an animal to m

In [27]:
chain.invoke("What is the meaning of Governmant?")

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


' The provided text does not directly answer your question about the meaning of "Government." However, it discusses a section of the Penal Code 1860 which describes an offense related to bribery in public service. Therefore, I will provide you with the definition of Government based on a separate source.\n\n   The term "Government" can refer to several meanings:\n\n   1. A political system in which power is concentrated in a group or individual who make decisions and exercise authority over a population or territory.\n\n   2. The administrative body that governs a country, state, or region, made up of elected or appointed officials.\n\n   3. In a broader context, it can also refer to the institutions through which a community organizes itself for collective decision-making and the enforcement of shared norms and values.'

In [28]:
chain.invoke("What will be the punishment for killing a person?")

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


' The Penal Code does not explicitly state the punishment for killing a person in the provided text. However, it is evident that certain acts causing death can be considered offenses if they are intended to cause death or if death is a foreseeable consequence and the act is not done in good faith for the benefit of another person. The severity of the punishment depends on the specific offense under the Code and whether it is punishable with imprisonment for a term of six months or upwards, as mentioned in the explanatory notes of sections 141, 176, 177, 201, 202, 212, 216, and 441. For more detailed information on the specific punishments for various offenses under the Penal Code, please refer to the original legislation or consult a legal professional.'

In [ ]:
vector_db.delete_collection()